In [19]:
from torch import nn
import numpy as np
import torch
from torch.autograd import Function
from torch.autograd import Variable

# Setup

### Parameters

In [20]:
batch_size_train = 200
batch_size_test = 1000
D = 784

### Load MNIST data

In [21]:
import torch.utils.data
from torchvision import datasets, transforms
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('MNIST_data', train=True, download=True,
               transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ])),
    batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('MNIST_data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size_test, shuffle=True)

# Train Feature Selector

In [22]:
def acc_fn(x1, x2, y1, y2):
    predictions = torch.sign(torch.sum(x1 * x2, dim=1))
    similarities = torch.sign(torch.eq(y1, y2).double() - 0.5)
    return torch.mean(torch.eq(predictions, similarities).double())

def hinge_loss_fn(x1, x2, y1, y2, gamma=0.5):    
    inner_products = torch.sum(x1 * x2, dim=1)
    same_class = torch.sign(torch.eq(y1, y2).double() - 0.5)
    scores = torch.clamp(gamma - same_class * inner_products, min=0)
    return torch.mean(scores)

def logistic_loss_fn(x1, x2, y1, y2):
    inner_products = torch.sum(x1 * x2, dim=1)
    same_class = torch.sign(torch.eq(y1, y2).double() - 0.5)
    scores = torch.log1p(torch.exp(-same_class * inner_products))
    return torch.mean(scores)

In [23]:
def features_train(epoch, shape):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data1, data2 = Variable(data).view(shape).chunk(2)
        classes1, classes2 = Variable(target).view(data.shape[0], 1).chunk(2)
        optimizer.zero_grad()
        features1, features2 = model(data1).double(), model(data2).double()
        loss = loss_fn(features1, features2, classes1, classes2)
        loss.backward()
        optimizer.step()        
        if batch_idx % 25 == 0:
            acc = acc_fn(features1, features2, classes1, classes2)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0], acc.data[0]))

def features_test(shape):
    model.eval()
    loss, acc, total = 0, 0, 0
    for data, target in test_loader:
        data1, data2 = Variable(data).view(shape).chunk(2)
        classes1, classes2 = Variable(target).view(data.shape[0], 1).chunk(2)
        optimizer.zero_grad()
        features1, features2 = model(data1).double(), model(data2).double()
        loss += loss_fn(features1, features2, classes1, classes2)
        acc += acc_fn(features1, features2, classes1, classes2)
        total += 1

    loss /= total
    acc /= total
    print('Test set: Loss: {:.4f}, Accuracy: {:.4f}%'.format(loss.data[0], 100. * acc.data[0]))
    return acc.data[0]

# Models

### 3 features, 3-layer NN

In [24]:
for lr in [0.0001]:
    for wd in [0.001]:
        loss_fn = hinge_loss_fn
        model = torch.nn.Sequential(
            torch.nn.Linear(D, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 3)
        )   
        optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay=wd)
        print('hinge loss | lr: {}, weight_decay: {}'.format(lr, wd))
        for epoch in range(8):
            features_train(epoch, (batch_size_train, -1))
            features_test((batch_size_test, -1))
        print()
dense = model

hinge loss | lr: 0.0001, weight_decay: 0.001
Train Epoch: 0 [0/60000 (0%)]	Loss: 0.585343	Accuracy: 0.172000
Train Epoch: 0 [5000/60000 (8%)]	Loss: 0.555307	Accuracy: 0.207600
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.537236	Accuracy: 0.327800
Train Epoch: 0 [15000/60000 (25%)]	Loss: 0.515610	Accuracy: 0.402000
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.506676	Accuracy: 0.474200
Train Epoch: 0 [25000/60000 (42%)]	Loss: 0.505098	Accuracy: 0.453200
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.509672	Accuracy: 0.507600
Train Epoch: 0 [35000/60000 (58%)]	Loss: 0.502643	Accuracy: 0.475400
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.509949	Accuracy: 0.476000
Train Epoch: 0 [45000/60000 (75%)]	Loss: 0.500553	Accuracy: 0.476000
Train Epoch: 0 [50000/60000 (83%)]	Loss: 0.502737	Accuracy: 0.460000
Train Epoch: 0 [55000/60000 (92%)]	Loss: 0.502578	Accuracy: 0.449600
Test set: Loss: 0.4980, Accuracy: 49.7886%
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.502249	Accuracy: 0.454400
Train Epoch: 1 [5000/60000 

In [32]:
class conv_net(nn.Module):
    def __init__(self, num_classes=1000):
        super(conv_net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 32, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64 * 10 * 10, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 3),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 64 * 10 * 10)
        x = self.classifier(x)
        return x
model = conv_net()
loss_fn = hinge_loss_fn
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001, weight_decay=1)
for epoch in range(5):
    features_train(epoch, (batch_size_train, 1, 28, 28))
    features_test((batch_size_test, 1, 28, 28))
print()
conv = model

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.503746	Accuracy: 0.120000
Train Epoch: 0 [5000/60000 (8%)]	Loss: 0.502998	Accuracy: 0.160000
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.503127	Accuracy: 0.116000
Train Epoch: 0 [15000/60000 (25%)]	Loss: 0.502551	Accuracy: 0.100000
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.502409	Accuracy: 0.090000
Train Epoch: 0 [25000/60000 (42%)]	Loss: 0.502317	Accuracy: 0.068800
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.501889	Accuracy: 0.120000
Train Epoch: 0 [35000/60000 (58%)]	Loss: 0.501781	Accuracy: 0.098200
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.501801	Accuracy: 0.110000
Train Epoch: 0 [45000/60000 (75%)]	Loss: 0.501638	Accuracy: 0.127600
Train Epoch: 0 [50000/60000 (83%)]	Loss: 0.501580	Accuracy: 0.110000
Train Epoch: 0 [55000/60000 (92%)]	Loss: 0.501322	Accuracy: 0.130000
Test set: Loss: 0.5009, Accuracy: 9.7600%
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.501194	Accuracy: 0.140000
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.501265	Accuracy: 0.090000
Train 

# Train linear classifier

In [ ]:
import tensorflow as tf
from sklearn.svm import LinearSVC
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [30]:
svc = LinearSVC(random_state=0)
# features_train = model(Variable(torch.from_numpy(train_data.reshape(train_data.shape[0],1,28,28)))).data.numpy()
f = model(Variable(torch.from_numpy(test_data.reshape(test_data.shape[0],1,28,28)))).data.numpy()
svc.fit(f, test_labels)
# print("train accuracy: {:.4f}".format(svc.score(features_train, train_labels)))
print("test accuracy: {:.4f}".format(svc.score(f, test_labels)))

test accuracy: 0.1135
